In [1]:
from pytorch_tabnet.tab_model import TabNetClassifier
import numpy as np
import pandas as pd
import os.path as osp

In [2]:
data_dir = '../data/'
df = pd.read_csv('../data/processed/train.csv', index_col='PassengerId')
df

,survived,pclass,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,familysz,cat_familysz,age_group
PassengerId,,,,,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,1,medium,young
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,1,medium,adult
3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,0,zero,young
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,1,medium,adult
5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,0,zero,adult
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S,0,zero,young
888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S,0,zero,teen
889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S,3,medium,NaN


# Complejidad 3 (30%)

## 12. Desarrolle un modelo que permita obtener los valores de la variable objetivo

### Data Preparation

In [3]:
continuous = ['age', 'fare', 'familysz', 'sibsp', 'parch']

categorical = [
    'pclass', 'sex', 'cat_familysz', 
    'age_group', 'embarked'
]

features = continuous + categorical

#### Delete duplicate rows

In [4]:
non_duplicates = df[features].drop_duplicates().index

#### Deal with missing values

In [5]:
from sklearn.model_selection import KFold

kf = KFold(n_splits=5, shuffle=True, random_state=42)
df[features].isna().sum()

age             177
fare              0
familysz          0
sibsp             0
parch             0
pclass            0
sex               0
cat_familysz      0
age_group       177
embarked          2
dtype: int64

In [18]:
kf.get_n_splits

<bound method _BaseKFold.get_n_splits of KFold(n_splits=5, random_state=42, shuffle=True)>

In [ ]:
df['cabin'].value_counts()

B96 B98        4
G6             4
C23 C25 C27    4
C22 C26        3
F33            3
              ..
E34            1
C7             1
C54            1
E36            1
C148           1
Name: cabin, Length: 147, dtype: int64

### Dataset split

### KNN Imputation

In [ ]:
from sklearn.impute import KNNImputer